## MLflow Tracking Quickstart

# Side comment from the teacher about what we are doing with tracking metrics:
- you need context of what we are doing but in short, we log the runs and in grafana we have a sql query that makes series from all the runs based on a metric, i.e binary cross entropy. We log it every day and make series out of it from a sql query
- good if we run 50-200 models one per day, bad if we want to scale it.

This notebook demonstrates using a local MLflow Tracking Server to log, register, and then load a model as a generic Python Function (pyfunc) to perform inference on a Pandas DataFrame.

Throughout this notebook, we'll be using the MLflow fluent API to perform all interactions with the MLflow Tracking Server.

In [1]:
import pandas as pd
from sklearn import datasets
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split

import mlflow
from mlflow.models import infer_signature

### Set the MLflow Tracking URI 

Depending on where you are running this notebook, your configuration may vary for how you initialize the interface with the MLflow Tracking Server. 

For this example, we're using a locally running tracking server, but other options are available (The easiest is to use the free managed service within [Databricks Community Edition](https://community.cloud.databricks.com/)). 

Please see [the guide to running notebooks here](https://www.mlflow.org/docs/latest/getting-started/running-notebooks/index.html) for more information on setting the tracking server uri and configuring access to either managed or self-managed MLflow tracking servers.

run mlflow ui --port 8080 in CLI (conda env)

In [2]:
# NOTE: review the links mentioned above for guidance on connecting to a managed tracking server, such as the free Databricks Community Edition

mlflow.set_tracking_uri(uri="http://127.0.0.1:5000")

## Load training data and train a simple model

For our quickstart, we're going to be using the familiar iris dataset that is included in scikit-learn. Following the split of the data, we're going to train a simple logistic regression classifier on the training data and calculate some error metrics on our holdout test data. 

Note that the only MLflow-related activities in this portion are around the fact that we're using a `param` dictionary to supply our model's hyperparameters; this is to make logging these settings easier when we're ready to log our model and its associated metadata.

In [3]:
# Load the Iris dataset
X, y = datasets.load_iris(return_X_y=True)

# Split the data into training and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Define the model hyperparameters
params = {"solver": "lbfgs", "max_iter": 1000, "multi_class": "auto", "random_state": 8888}

# Train the model
lr = LogisticRegression(**params)
lr.fit(X_train, y_train)

# Predict on the test set
y_pred = lr.predict(X_test)

# Calculate accuracy as a target loss metric
accuracy = accuracy_score(y_test, y_pred)

/home/bla/.local/share/virtualenvs/mlops-0su7QrrH/lib/python3.10/site-packages/sklearn/linear_model/_logistic.py:1247: FutureWarning: 'multi_class' was deprecated in version 1.5 and will be removed in 1.7. From then on, it will always use 'multinomial'. Leave it to its default value to avoid this warning.
  warnings.warn(


## Define an MLflow Experiment

In order to group any distinct runs of a particular project or idea together, we can define an Experiment that will group each iteration (runs) together. 
Defining a unique name that is relevant to what we're working on helps with organization and reduces the amount of work (searching) to find our runs later on. 

In [4]:
mlflow.set_experiment("MLflow Quickstart")

<Experiment: artifact_location='mlflow-artifacts:/824395778428720242', creation_time=1725524034548, experiment_id='824395778428720242', last_update_time=1725524034548, lifecycle_stage='active', name='MLflow Quickstart', tags={}>

## Log the model, hyperparameters, and loss metrics to MLflow.

In order to record our model and the hyperparameters that were used when fitting the model, as well as the metrics associated with validating the fit model upon holdout data, we initiate a run context, as shown below. Within the scope of that context, any fluent API that we call (such as `mlflow.log_params()` or `mlflow.sklearn.log_model()`) will be associated and logged together to the same run. 

In [5]:
# Start an MLflow run
with mlflow.start_run():
    # Log the hyperparameters
    mlflow.log_params(params)

    # Log the loss metric
    mlflow.log_metric("accuracy", accuracy)

    # Set a tag that we can use to remind ourselves what this run was for
    mlflow.set_tag("Training Info", "Basic LR model for iris data")

    # Infer the model signature
    signature = infer_signature(X_train, lr.predict(X_train))

    # Log the model
    model_info = mlflow.sklearn.log_model(
        sk_model=lr,
        artifact_path="iris_model",
        signature=signature,
        input_example=X_train,
        registered_model_name="tracking-quickstart",
    )

Registered model 'tracking-quickstart' already exists. Creating a new version of this model...
2024/09/05 13:50:07 INFO mlflow.store.model_registry.abstract_store: Waiting up to 300 seconds for model version to finish creation. Model name: tracking-quickstart, version 2
Created version '2' of model 'tracking-quickstart'.


2024/09/05 13:50:07 INFO mlflow.tracking._tracking_service.client: 🏃 View run welcoming-hawk-698 at: http://127.0.0.1:5000/#/experiments/824395778428720242/runs/3b25987f05f149aca52860307932443b.
2024/09/05 13:50:07 INFO mlflow.tracking._tracking_service.client: 🧪 View experiment at: http://127.0.0.1:5000/#/experiments/824395778428720242.


## Load our saved model as a Python Function

Although we can load our model back as a native scikit-learn format with `mlflow.sklearn.load_model()`, below we are loading the model as a generic Python Function, which is how this model would be loaded for online model serving. We can still use the `pyfunc` representation for batch use cases, though, as is shown below.

In [6]:
loaded_model = mlflow.pyfunc.load_model(model_info.model_uri)

In [7]:
print(loaded_model)

mlflow.pyfunc.loaded_model:
  artifact_path: iris_model
  flavor: mlflow.sklearn
  run_id: 3b25987f05f149aca52860307932443b



## Use our model to predict the iris class type on a Pandas DataFrame

In [8]:
predictions = loaded_model.predict(X_test)

iris_feature_names = datasets.load_iris().feature_names

# Convert X_test validation feature data to a Pandas DataFrame
result = pd.DataFrame(X_test, columns=iris_feature_names)

# Add the actual classes to the DataFrame
result["actual_class"] = y_test

# Add the model predictions to the DataFrame
result["predicted_class"] = predictions

result[:4]

,sepal length (cm),sepal width (cm),petal length (cm),petal width (cm),actual_class,predicted_class
0,6.1,2.8,4.7,1.2,1,1
1,5.7,3.8,1.7,0.3,0,0
2,7.7,2.6,6.9,2.3,2,2
3,6.0,2.9,4.5,1.5,1,1


## Listing and Searching MLflow Models
You can fetch a list of registered models in the registry with a simple method.

In [9]:
from mlflow import MlflowClient

from pprint import pprint

client = MlflowClient()
for rm in client.search_registered_models():
    pprint(dict(rm), indent=4)

{   'aliases': {},
    'creation_timestamp': 1725524037211,
    'description': '',
    'last_updated_timestamp': 1725537007023,
    'latest_versions': [   <ModelVersion: aliases=[], creation_timestamp=1725537007023, current_stage='None', description='', last_updated_timestamp=1725537007023, name='tracking-quickstart', run_id='3b25987f05f149aca52860307932443b', run_link='', source='mlflow-artifacts:/824395778428720242/3b25987f05f149aca52860307932443b/artifacts/iris_model', status='READY', status_message='', tags={}, user_id='', version='2'>],
    'name': 'tracking-quickstart',
    'tags': {}}


With hundreds of models, it can be cumbersome to peruse the results returned from this call. A more efficient approach would be to search for a specific model name and list its version details using search_model_versions() method and provide a filter string such as "name='tracking-quickstart'"

In [10]:
client = MlflowClient()
for mv in client.search_model_versions("name='tracking-quickstart'"):
    pprint(dict(mv), indent=4)

{   'aliases': [],
    'creation_timestamp': 1725537007023,
    'current_stage': 'None',
    'description': '',
    'last_updated_timestamp': 1725537007023,
    'name': 'tracking-quickstart',
    'run_id': '3b25987f05f149aca52860307932443b',
    'run_link': '',
    'source': 'mlflow-artifacts:/824395778428720242/3b25987f05f149aca52860307932443b/artifacts/iris_model',
    'status': 'READY',
    'status_message': '',
    'tags': {},
    'user_id': '',
    'version': '2'}
{   'aliases': [],
    'creation_timestamp': 1725524037223,
    'current_stage': 'None',
    'description': '',
    'last_updated_timestamp': 1725524037223,
    'name': 'tracking-quickstart',
    'run_id': '51118cba4ffd4d43b9b1e4eb3a7a5dd5',
    'run_link': '',
    'source': 'mlflow-artifacts:/824395778428720242/51118cba4ffd4d43b9b1e4eb3a7a5dd5/artifacts/iris_model',
    'status': 'READY',
    'status_message': '',
    'tags': {},
    'user_id': '',
    'version': '1'}


## Querying Runs Programmatically
Search runs of an experiment

In [11]:
complex_filter = f"""
  metrics.accuracy > 0.9
"""

runs_with_complex_filter = mlflow.search_runs(
    experiment_names=["MLflow Quickstart"],
    filter_string=complex_filter,
)
print(runs_with_complex_filter)

                             run_id       experiment_id    status  \
0  3b25987f05f149aca52860307932443b  824395778428720242  FINISHED   
1  51118cba4ffd4d43b9b1e4eb3a7a5dd5  824395778428720242  FINISHED   

                                        artifact_uri  \
0  mlflow-artifacts:/824395778428720242/3b25987f0...   
1  mlflow-artifacts:/824395778428720242/51118cba4...   

                        start_time                         end_time  \
0 2024-09-05 11:50:05.005000+00:00 2024-09-05 11:50:07.297000+00:00   
1 2024-09-05 08:13:54.652000+00:00 2024-09-05 08:13:57.704000+00:00   

   metrics.accuracy params.max_iter params.random_state params.multi_class  \
0               1.0            1000                8888               auto   
1               1.0            1000                8888               auto   

  params.solver  tags.mlflow.runName            tags.Training Info  \
0         lbfgs   welcoming-hawk-698  Basic LR model for iris data   
1         lbfgs  auspicious-dove-1

More info on Search Runs: https://mlflow.org/docs/latest/search-runs.html

## Deleting MLflow Models
You can either delete specific versions of a registered model or you can delete a registered model and all its versions.

In [14]:
# Delete versions 1,2, and 3 of the model
client = MlflowClient()
versions = [2]
for version in versions:
    client.delete_model_version(
        name="tracking-quickstart", version=version
    )

RestException: RESOURCE_DOES_NOT_EXIST: Model Version (name=tracking-quickstart, version=2) not found

More info on MLflow Model Registry:
https://mlflow.org/docs/latest/model-registry.html